# Loading A01T

In [1]:
import autoreload

For training segment, as done in the FBCSP algorithm only using data of each trial in between 0.5 and 2.5 seconds after cue onset

In [2]:
import numpy as np
fs = 250

data = np.load('Dataset 2A/A01T.npz')
signals = data['s']

eeg = signals.T[:-3]

n_trials = 288

In [3]:
event_types = data['etyp']
event_durations = data['edur']
event_positions = data['epos']

training_trials = {
    769: [],
    770: [],
    771: [],
    772: []
}

cue_onset_codes = [769,770,771,772]

for i in range(len(event_types)):

    event_code = event_types[i][0]
    event_position = event_positions[i][0]

    if event_code in cue_onset_codes:

        start_idx = event_position + int(0.5*fs)
        end_idx = start_idx + int(2*fs)

        cur_trial = eeg[:,start_idx:end_idx]
        training_trials[event_code].append(cur_trial)


In [4]:
trials = np.ones((n_trials,len(eeg),fs*2))
labels = np.array([1]*72 + [2]*72 + [3]*72 + [4]*72)

idx_counter = 0

for class_label in training_trials:
    for trial in training_trials[class_label]:
        trials[idx_counter] = trial

# Filter Banks

In [5]:
from fbcsp_code.bin.MLEngine import FilterBank

In [6]:
filter_bank = FilterBank(fs)
filter_coefs = filter_bank.get_filter_coeff()

filter_bank_trials = filter_bank.filter_data(trials)

# Approach 1 - Divide and Conquer

In [7]:
class_labels = set(labels)


#i.e different datasets for the classifiers

'''
    class one versus class 2,3,4
    class two versus 3,4
    class three versus 4

    if we get past classifier for class three versus 4, then auto assign class 4 as prediction
'''

divide_and_conquer_datasets = {
    0:{
        'class_one_trials':[],
        'class_two_trials':[]
    },
    1:{
        'class_one_trials':[],
        'class_two_trials':[]
    },
    2:{
        'class_one_trials':[],
        'class_two_trials':[]
    }
}

possible_label_combinations = [
    [[1],[2,3,4]], [[2],[3,4]], [[3],[4]]
]

for i,label_combination in enumerate(possible_label_combinations):

    class_one_labels = label_combination[0]
    class_two_labels = label_combination[1]

    for j,label in enumerate(labels):
        
        cur_trial = filter_bank_trials[:,j]
        if label in class_one_labels:
            divide_and_conquer_datasets[i]['class_one_trials'].append(cur_trial)

        elif label in class_two_labels:
            divide_and_conquer_datasets[i]['class_two_trials'].append(cur_trial)



Now that we have each of the datasets, we fit CSP to each one and get joint diagonalization of the covariance matrices for the combined class distributions

In [8]:
from fbcsp_code.bin.CSP import CSP

In [9]:
initial_feature_vecs = {
    0:{
        'class_one_trials':[],
        'class_two_trials':[]
    },
    1:{
        'class_one_trials':[],
        'class_two_trials':[]
    },
    2:{
        'class_one_trials':[],
        'class_two_trials':[]
    }
}


for key in divide_and_conquer_datasets:

    class_one_trials = divide_and_conquer_datasets[key]['class_one_trials']
    class_two_trials = divide_and_conquer_datasets[key]['class_two_trials']
    
    
    class_labels = [0]*len(class_one_trials) + [1]*len(class_two_trials)
    class_trials = np.concatenate((class_one_trials,class_two_trials)).transpose(1,0,2,3)
    
    n_trials = class_trials.shape[1]
    
    csp_projection_matrices = []
    
    for i,frequency_band in enumerate(class_trials):
        cur_trials = class_trials[i]
        
        cur_csp_filter = CSP(m_filters = 2)
        cur_eig_vals,cur_eig_vectors = cur_csp_filter.fit(cur_trials,class_labels)
        csp_projection_matrices.append(cur_eig_vectors)
    
    
    
    csp_filter = CSP(m_filters = 2)
    V_bar = []
    for i in range(n_trials):
    
        v_i = np.array([])
        for j,projection_matrix in enumerate(csp_projection_matrices):
    
            cur_trial = class_trials[j][i]
            v_b_i = csp_filter.transform(cur_trial,projection_matrix)
            v_i = np.concatenate((v_i,v_b_i))
    

        cur_label = class_labels[i]

        if cur_label == 0:
            initial_feature_vecs[key]['class_one_trials'].append(v_i)
        elif cur_label == 1:
            initial_feature_vecs[key]['class_two_trials'].append(v_i)
    


In [10]:
from fbcsp_code.bin.Classifier import FeatureSelect

In [13]:
feature_selector = FeatureSelect()

class_one_trials = initial_feature_vecs[0]['class_one_trials']
class_two_trials = initial_feature_vecs[0]['class_two_trials']

train_trials = np.concatenate((class_one_trials,class_two_trials))
train_labels = [0]*len(class_one_trials) + [1]*len(class_two_trials)

training_features = feature_selector.fit(train_trials,train_labels)

(288, 8)
